# Node.js pragmatics

## Command-line arguments

Command-line arguments are contained in the `argv` property of the global `process` object. The `argv` property is an array with the arguments, represented as strings, beginning at index 2:

In [ ]:
['node', '/path/to/your/program.js', 'arg1', 'arg2', 'arg3'...]

Iterating through `process.argv`:

In [1]:
// Simulate running a script file from the command line 
// with three numeric arguments
process.argv = ['node', 'app.js', '1', '2', '3'];

// Contents of imagined script file app.js
var sum = 0;
for (var i = 2, l = process.argv.length; i < l; i ++) {

    // process.argv elements are strings, so we coerce to numbers here
    sum += Number(process.argv[i]);
}
console.log(sum); // => 6

6


undefined

On the first line above, we set `process.argv` to an array that simulates running a script file from the command line with three numeric arguments, as if we wrote this at a bash prompt:

`$ node app.js 1 2 3`

The code that follows the first line above would reside in that script file `app.js`.

## File system in/out

For file system input, require the `fs` module and call its `readFile()` methods.

### Synchronous (blocking) method

In [ ]:
var fs = require('fs');
var myBufferObject = fs.readFileSync('./myFile.txt'); // filepath

Stringifying the `Buffer` object returned by the `readFile()` methods of the `fs` module:

In [ ]:
/* Read a file from a path specified as first command line argument
   and log number of lines in the file to the console */

// Simulate command-line argument
process.argv = ['node', 'app.js', './fileSystemInOut.txt'];

// Require 'fs' (filesystem) module
var fs = require('fs');

/* Use synchronous (blocking) method. Returns a `Buffer` object.
   Remember that first CLI arg is `process.argv` index 2, not 0 */
var myBufferObject = fs.readFileSync(process.argv[2]);

/* Stringify the `Buffer` object and call `.split('\n')` to create an
   array of elements separated by newlines */
var myBufferObjectStringified = myBufferObject.toString();
var myBufferObjectStringifiedArray = 
    myBufferObjectStringified.split('\n');

/* Use length of the array created by `.split()`
   => "./fileSystemInOut.txt contains 5 lines." */
console.log(process.argv[2] + ' contains ' + 
            myBufferObjectStringifiedArray.length + ' lines.');

More tersely:

In [ ]:
var fs = require('fs')

var contents = fs.readFileSync(process.argv[2])
var lines    = contents.toString().split('\n').length

Also, the `fs` module's `readFile()` methods take an optional `encoding` parameter which will stringify the `Buffer` object, eliminating the need to call `.toString()`:

In [ ]:
var lines = fs.readFileSync(process.argv[2], 'utf8').split('\n').length;

### Asynchronous (non-blocking) method

Requires a callback. Here, we also check `error`:

In [ ]:
var fs = require('fs');

var myBufferObject = fs.readFile('./myFile.txt', 'utf-8',
        function(error, contents) {
            if (error) throw error;
            console.log(contents);
        }
    );

### Reading directory contents

#### Filter list of files in directory by extension

In [ ]:
// Simulate command-line argument
process.argv = ['node', './app.js', './', 'js'];

var fs = require('fs');

// `Path` module provides `extname()` method we will use
var path = require('path');

// Call the `fs` module's `readdir()` method
fs.readdir(process.argv[2], function(error, list) {
    if (error) throw error;

    for (var i = 0, l = list.length; i < l; i++) {

        // `path.extname()` returns file extension including '.'
        if (path.extname(list[i]) === '.' + process.argv[3]) {
            console.log(list[i]);
        }
    }
});

This could also be accomplished using the `forEach()` method:

In [ ]:
var fs = require('fs')
var path = require('path')

fs.readdir(process.argv[2], function (err, list) {
  list.forEach(function (file) {
    if (path.extname(file) === '.' + process.argv[3])
      console.log(file)
  })
})

## Serving

For a basic HTTP server, require the `http` module and call its `createServer()` method, passing it a callback:

In [ ]:
var http = require('http');

http.createServer(function(request, response) {
  response.writeHead(200);
  response.end();
}).listen(8080);

Serving the contents of a file from the filesystem:

In [ ]:
var http = require('http');
var fs   = require('fs');

http.createServer(function(request, response) {
  response.writeHead(200);
  fs.readFile('index.html', function(error, contents) {
    response.write(contents);
    response.end(); // End response here
  });
}).listen(8080);

If this code is in a file named `server.js`, run it from the command line with `node server.js`, then use `curl http://localhost:8080` to see the server response.

Writing HTTP response headers along with a status code:

In [ ]:
response.writeHead(200, {
    'Content-Type': 'text/html',
    ...
});

`response.end()` can take a parameter the value of which will be appended to the response:

In [ ]:
var http = require('http');
var fs   = require('fs');

http.createServer(function(request, response) {
  response.writeHead(200);
  fs.readFile('index.html', function(error, contents) {
    response.write(contents);
    response.end("Goodbye");
  });
}).listen(8080);

In [ ]:
$ echo "foo" > index.html
$ node server.js
$ curl http://localhost:8080
foo
Goodbye

## Modules

Exporting a single function from a module (the callback should be placed inside the appropriate block, and one should perform an error check somewhere using `if (error) return callback(error);`):

In [ ]:
module.exports = function(param1, param2, callback) {
    ...
    callback(null, fileArray);
};

Using the module with the callback:

In [ ]:
var myModule = require('myModule.js'); // <- looks in node_modules directory;
                                       // or you can use Unix pathnames

myModule(param1, param1, function(error, data) {
    if (error) return console.error('Error: ' + error);
    ...
});

Exporting multiple functions from a module:

In [ ]:
exports.function1 = function() {};
exports.function2 = function() {};

In [ ]:
var myModule = require('myModule.js');
myModule.function1();
myModule.function2();

## Streams (stream API still has status Unstable in node v0.12.6)

Create read file stream and log chunks to console:

In [ ]:
var fs = require('fs');

var file = fs.createReadStream('myFile.txt');
file.on('readable', function() {
    var chunk;
    while (null !== (chunk = file.read())) {
        console.log(chunk.toString());
      }
});

Using `pipe()` method instead:

In [ ]:
var fs = require('fs');

var file = fs.createReadStream('myFile.txt');

file.pipe(process.stdout);